In [ ]:
# для colab
import os, sys, subprocess

REPO = "andersonTheCat/witch_practicum"
BRANCH = "main"
DEST = "/content/_repo"

if not os.path.exists(DEST):
    subprocess.run(
        ["git", "clone", "--depth", "1", "-b", BRANCH, f"https://github.com/{REPO}.git", DEST],
        check=True
    )
else:
    subprocess.run(["git", "-C", DEST, "pull", "--ff-only"], check=True)

if DEST not in sys.path:
    sys.path.insert(0, DEST)

print("(=^..^=) repo ready at", DEST)

In [41]:
import numpy as np
from IPython.display import display, Markdown

from meowmeow import (
    Mat, Vec, display_latex, _err,
    _as_array_M, _as_array_v,
)
from meowmeowmeow import _vnorm

from meowmeowmeowmeow import power_normalized, make_symmetric_with_spectrum, make_x0_eig

In [42]:
def _rayleigh(A: np.ndarray, x: np.ndarray):
    num = np.vdot(x, A @ x)
    den = np.vdot(x, x)
    if den == 0:
        raise _err("в отношении Рэлея (x,x)=0 - это неуютно")
    return num / den

In [43]:
def _fmt_num(z):
    z = complex(z)
    if abs(z.imag) < 1e-12:
        return f"{z.real:.12g}"
    return f"{z.real:.12g} + {z.imag:.12g}i"

In [44]:
def _safe_cond2(M: np.ndarray):
    try:
        return float(np.linalg.cond(M, 2))
    except Exception:
        return float("nan")

In [45]:
def _solve_linear(M: np.ndarray, rhs: np.ndarray):
    try:
        return np.linalg.solve(M, rhs), None
    except np.linalg.LinAlgError:
        y = np.linalg.lstsq(M, rhs, rcond=None)[0]
        return y, "LinAlgError в solve, использовали lstsq (на всякий случай)"

In [46]:
def _print_basic_stats(A: Mat):
    Aarr = _as_array_M(A)
    display(Markdown("**матрица $A$**"))
    display_latex(A, label=r"A")
    display(Markdown(f"cond2(A) ≈ `{_safe_cond2(Aarr):.6g}`"))

In [47]:
def _show_stop_rules_formulas_once():
    display(Markdown(
        "**критерии остановки:**  \n"
        "`residual`: $\\sigma=\\dfrac{\\|Ax-\\lambda x\\|}{\\|x\\|}<\\varepsilon$  \n"
        "`delta`: $\\|x^{(k)}-x^{(k-1)}\\|<\\varepsilon$  \n"
        "`lambda`: $\\big|\\lambda^{(k)}-\\lambda^{(k-1)}\\big|<\\varepsilon$"
    ))

In [48]:
def _ask_choice(prompt: str, allowed: set, default: str):
    s = (input(prompt).strip().lower() or default).strip().lower()
    if s not in allowed:
        raise _err(f"неуютный выбор `{s}`; допустимо: {', '.join(sorted(allowed))}")
    return s

In [49]:
def rayleigh_inverse_iterations(A_in: Mat,
                               x0: Vec,
                               eps: float = 1e-12,
                               max_iter: int = 20,
                               stop_rule: str = "residual",
                               vec_norm_kind: str = "2",
                               log_each: bool = True):
    A = _as_array_M(A_in)
    n, m = A.shape
    if n != m:
        raise _err("нужна квадратная матрица A")

    x = _as_array_v(x0).astype(complex).copy()
    if x.shape != (n,):
        raise _err("x0 должен иметь размерность (n,)")
    nx = np.linalg.norm(x)
    if nx == 0:
        raise _err("x0 не должен быть нулевым")
    x = x / nx

    lam_old = None
    history = []

    if log_each:
        display(Markdown(
            "**обратные итерации с отношением Рэлея:**  \n"
            f"eps=`{eps}`, max_iter=`{max_iter}`, stop=`{stop_rule}`"
        ))
        display_latex(Vec(x.real if np.max(np.abs(x.imag)) < 1e-12 else x), label=r"x^{(0)}")

    for k in range(1, max_iter + 1):
        lam = _rayleigh(A, x)
        M = A - lam * np.eye(n, dtype=complex)
        cM = _safe_cond2(M)

        y, warn = _solve_linear(M, x)
        ny = np.linalg.norm(y)
        if ny == 0:
            raise _err("получилось y=0, нормировка невозможна")
        x_new = y / ny

        lam_new = _rayleigh(A, x_new)
        r = (A @ x_new) - lam_new * x_new
        sigma = _vnorm(r, vec_norm_kind) / (_vnorm(x_new, vec_norm_kind) + 1e-300)

        dlam = None if lam_old is None else abs(lam_new - lam_old)

        extra = {}
        if stop_rule == "residual":
            ok = (sigma < eps)
        elif stop_rule == "lambda":
            ok = (dlam is not None) and (dlam < eps)
            if dlam is not None:
                extra["dlam"] = float(np.abs(dlam))
        else:
            raise _err("stop_rule (rayleigh inverse): residual / lambda")

        history.append({
            "k": k,
            "lambda": complex(lam_new),
            "sigma": float(sigma),
            "cond_shift": float(cM),
            "warn": warn,
            **extra
        })

        if log_each:
            msg = (
                f"**шаг {k}:**  "
                f"λ≈`{_fmt_num(lam_new)}`,  "
                f"sigma≈`{sigma:.6g}`,  "
                f"cond2(A-λI)≈`{cM:.6g}`"
            )
            if dlam is not None:
                msg += f",  |Δλ|≈`{float(np.abs(dlam)):.6g}`"
            if warn:
                msg += f"  ({warn})"
            display(Markdown(msg))
            display_latex(Vec(x_new.real if np.max(np.abs(x_new.imag)) < 1e-12 else x_new),
                          label=rf"\hat e^{({k})}")

        lam_old = lam_new
        x = x_new
        if ok:
            return Vec(x.real if np.max(np.abs(x.imag)) < 1e-12 else x), {
                "iters": k,
                "lambda": complex(lam_new),
                "sigma": float(sigma),
                "history": history,
                "method": "rayleigh_inverse",
                "stop_rule": stop_rule
            }

    lam_new = _rayleigh(A, x)
    r = (A @ x) - lam_new * x
    sigma = _vnorm(r, vec_norm_kind) / (_vnorm(x, vec_norm_kind) + 1e-300)
    return Vec(x.real if np.max(np.abs(x.imag)) < 1e-12 else x), {
        "iters": max_iter,
        "lambda": complex(lam_new),
        "sigma": float(sigma),
        "history": history,
        "method": "rayleigh_inverse",
        "stop_rule": stop_rule,
        "warn": "достигнут max_iter, но критерий не сработал"
    }

In [50]:
def inverse_iterations(A_in: Mat, lam_star,
                       x0: Vec,
                       eps: float = 1e-6,
                       max_iter: int = 30,
                       stop_rule: str = "residual",
                       vec_norm_kind: str = "2",
                       log_each: bool = True):
    A = _as_array_M(A_in)
    n, m = A.shape
    if n != m:
        raise _err("нужна квадратная матрица A")

    x = _as_array_v(x0).astype(complex).copy()
    if x.shape != (n,):
        raise _err("x0 должен иметь размерность (n,)")
    nx = np.linalg.norm(x)
    if nx == 0:
        raise _err("x0 не должен быть нулевым")
    x = x / nx

    lam_star = complex(lam_star)
    M = A - lam_star * np.eye(n, dtype=complex)
    cM = _safe_cond2(M)

    if log_each:
        display(Markdown(
            "**метод обратных итераций (фиксированное $\\lambda^*$):**  \n"
            f"$\\lambda^*$ = `{_fmt_num(lam_star)}`  \n"
            f"cond2(A-λ*I) ≈ `{cM:.6g}`  \n"
            f"eps=`{eps}`, max_iter=`{max_iter}`, stop=`{stop_rule}`"
        ))
        display_latex(Vec(x.real if np.max(np.abs(x.imag)) < 1e-12 else x), label=r"x^{(0)}")

    history = []
    for k in range(1, max_iter + 1):
        y, warn = _solve_linear(M, x)
        ny = np.linalg.norm(y)
        if ny == 0:
            raise _err("получилось y=0, нормировка невозможна")
        x_new = y / ny

        lam_hat = _rayleigh(A, x_new)
        r = (A @ x_new) - lam_hat * x_new
        sigma = _vnorm(r, vec_norm_kind) / (_vnorm(x_new, vec_norm_kind) + 1e-300)
        delta = _vnorm(x_new - x, vec_norm_kind)

        if stop_rule == "residual":
            ok = (sigma < eps)
        elif stop_rule == "delta":
            ok = (delta < eps)
        else:
            raise _err("stop_rule: residual / delta")

        history.append({
            "k": k,
            "lambda": complex(lam_hat),
            "sigma": float(sigma),
            "delta": float(delta),
            "warn": warn
        })

        if log_each:
            msg = (
                f"**шаг {k}:**  "
                f"λ_R(x)≈`{_fmt_num(lam_hat)}`,  "
                f"sigma≈`{sigma:.6g}`,  "
                f"delta≈`{delta:.6g}`"
            )
            if warn:
                msg += f"  ({warn})"
            display(Markdown(msg))
            display_latex(Vec(x_new.real if np.max(np.abs(x_new.imag)) < 1e-12 else x_new),
                          label=rf"\hat e^{({k})}")

        x = x_new
        if ok:
            return Vec(x.real if np.max(np.abs(x.imag)) < 1e-12 else x), {
                "iters": k,
                "lambda": complex(lam_hat),
                "sigma": float(sigma),
                "delta": float(delta),
                "lambda_star": complex(lam_star),
                "cond_shift": float(cM),
                "history": history,
                "method": "inverse_fixed",
                "stop_rule": stop_rule
            }

    lam_hat = _rayleigh(A, x)
    r = (A @ x) - lam_hat * x
    sigma = _vnorm(r, vec_norm_kind) / (_vnorm(x, vec_norm_kind) + 1e-300)
    return Vec(x.real if np.max(np.abs(x.imag)) < 1e-12 else x), {
        "iters": max_iter,
        "lambda": complex(lam_hat),
        "sigma": float(sigma),
        "delta": float("nan"),
        "lambda_star": complex(lam_star),
        "cond_shift": float(cM),
        "history": history,
        "method": "inverse_fixed",
        "stop_rule": stop_rule,
        "warn": "достигнут max_iter, но критерий не сработал"
    }

In [51]:
def _compare_with_numpy(A: Mat, lam_hat, e_hat: Vec):
    Aarr = _as_array_M(A)
    x = _as_array_v(e_hat).astype(complex)
    try:
        eigs = np.linalg.eigvals(Aarr)
        j = int(np.argmax(np.abs(eigs)))
        lam_true = eigs[j]
        r = Aarr @ x - complex(lam_hat) * x
        display(Markdown(
            "**проверка (numpy):**  \n"
            f"numpy max|λ| ≈ `{lam_true}`  \n"
            f"наш λ_hat ≈ `{_fmt_num(lam_hat)}`  \n"
            f"||Ax-λx||_2 ≈ `{np.linalg.norm(r):.6g}`"
        ))
    except Exception:
        display(Markdown("**проверка (numpy):** не удалось посчитать eigvals :<"))

In [52]:
def _make_x0(kind: str, n: int, seed: int = 42):
    if make_x0_eig is not None:
        return make_x0_eig(kind, n, seed=seed)
    k = (kind or "").strip().lower()
    if k in {"ones", "1", "ед", "единицы"}:
        return Vec(np.ones(n))
    if k in {"rand", "random", "сл", "случ"}:
        rng = np.random.default_rng(seed)
        x = rng.uniform(-1.0, 1.0, size=n)
        if np.linalg.norm(x) == 0:
            x[0] = 1.0
        return Vec(x)
    if k.startswith("e"):
        idx = int(k[1:]) - 1
        if not (0 <= idx < n):
            raise _err("x0 kind вида e1/e2/.../en")
        x = np.zeros(n)
        x[idx] = 1.0
        return Vec(x)
    raise _err("x0 kind: ones / rand / e1..en")

In [53]:
def _get_lambda_star_by_power(A: Mat, x0: Vec, eps=1e-6, max_iter=500, log_each=False):
    if power_normalized is None:
        lam = _rayleigh(_as_array_M(A), _as_array_v(x0).astype(complex))
        return lam, "power_normalized не найден, взяли λ* = ρ(x0)"
    _, info = power_normalized(A, x0, eps=eps, max_iter=max_iter, stop_rule="residual", log_each=log_each)
    return info["lambda"], None

In [54]:
def solve_one_case(A: Mat, x0: Vec,
                   eps_list=(1e-3, 1e-6),
                   want_power_lambda_star=True,
                   lambda_star_manual=None,
                   stop_rule_fixed="residual",
                   stop_rule_rayleigh="residual",
                   log_each=True):
    _print_basic_stats(A)

    n = _as_array_M(A).shape[0]
    display(Markdown(f"**параметры:**  \n n = `{n}`"))
    display(Markdown("**x0:**  \n варианты: `ones` / `rand` / `e1`..`en`"))
    display_latex(x0, label=r"x^{(0)}")

    if want_power_lambda_star:
        lam_star, warn = _get_lambda_star_by_power(A, x0, eps=1e-6, max_iter=500, log_each=False)
        if warn:
            display(Markdown(f"**заметка:** {warn}"))
    else:
        if lambda_star_manual is None:
            raise _err("нужно задать lambda_star_manual, если want_power_lambda_star=False")
        lam_star = complex(lambda_star_manual)

    display(Markdown(f"**$\\lambda^*$ для фиксированных обратных итераций:** `{_fmt_num(lam_star)}`"))

    for eps in eps_list:
        display(Markdown(f"**прогон для eps = `{eps}` (фиксированное $\\lambda^*$)**"))
        e_fix, info_fix = inverse_iterations(
            A, lam_star, x0, eps=eps, max_iter=30,
            stop_rule=stop_rule_fixed, log_each=log_each
        )
        msg = (
            f"**итог (фиксированное $\\lambda^*$):** "
            f"итераций = **{info_fix['iters']}**, "
            f"λ_R ≈ `{_fmt_num(info_fix['lambda'])}`, "
            f"sigma≈`{info_fix['sigma']:.6g}`"
        )
        if info_fix.get("warn"):
            msg += f"  ({info_fix['warn']})"
        display(Markdown(msg))
        display_latex(e_fix, label=r"\hat e \;(\mathrm{inverse})")
        _compare_with_numpy(A, info_fix["lambda"], e_fix)

        display(Markdown(f"**прогон для eps = `{eps}` (Рэлеевские обратные итерации)**"))
        e_r, info_r = rayleigh_inverse_iterations(
            A, x0,
            eps=min(1e-12, eps * 1e-2),
            max_iter=20,
            stop_rule=stop_rule_rayleigh,
            log_each=log_each
        )
        msg2 = (
            f"**итог (Рэлея):** "
            f"итераций = **{info_r['iters']}**, "
            f"λ ≈ `{_fmt_num(info_r['lambda'])}`, "
            f"sigma≈`{info_r['sigma']:.6g}`"
        )
        if info_r.get("warn"):
            msg2 += f"  ({info_r['warn']})"
        display(Markdown(msg2))
        display_latex(e_r, label=r"\hat e \;(\mathrm{rayleigh})")
        _compare_with_numpy(A, info_r["lambda"], e_r)

    print("мяу, готово (=^..^=)")

In [55]:
def demo_three_inputs_inverse(n=6):
    if make_symmetric_with_spectrum is None:
        display(Markdown("**демо:** генератор make_symmetric_with_spectrum не найден, пропускаем демо :<"))
        return

    display(Markdown("**демо 1:** |λ1| сильно больше |λ2| (должно быть очень быстро)"))
    A1 = make_symmetric_with_spectrum([5.0, 1.0] + [0.2] * (n - 2), seed=7)
    x0 = _make_x0("rand", n, seed=1)
    solve_one_case(A1, x0, eps_list=(1e-3, 1e-6), stop_rule_fixed="residual", stop_rule_rayleigh="residual", log_each=True)

    display(Markdown("**демо 2:** |λ2/λ1| почти 1 (должно быть помедленнее)"))
    A2 = make_symmetric_with_spectrum([1.0, 0.99] + [0.2] * (n - 2), seed=8)
    x0 = _make_x0("rand", n, seed=2)
    solve_one_case(A2, x0, eps_list=(1e-3, 1e-6), stop_rule_fixed="residual", stop_rule_rayleigh="residual", log_each=False)

    display(Markdown("**демо 3:** старт x0 близко к нужному направлению (для Рэлея это должно быть уютно)"))
    A3 = make_symmetric_with_spectrum([3.0, 1.5] + [0.1] * (n - 2), seed=9)
    x0 = _make_x0("e1", n, seed=3)
    solve_one_case(A3, x0, eps_list=(1e-6,), stop_rule_fixed="residual", stop_rule_rayleigh="residual", log_each=True)

In [56]:
def _read_matrix_fallback():
    display(Markdown(
        "**ввод матрицы A:**  \n"
        "вводи строки через пробел, пустая строка = конец  \n"
        "пример для 3x3:  \n"
        "`1 2 3`  \n"
        "`0 4 5`  \n"
        "`0 0 6`"
    ))
    rows = []
    while True:
        s = input().strip()
        if s == "":
            break
        rows.append([float(t) for t in s.split()])
    if not rows:
        raise _err("пустой ввод матрицы")
    m = len(rows[0])
    if any(len(r) != m for r in rows):
        raise _err("строки разной длины")
    A = np.array(rows, dtype=float)
    return Mat(A)

In [57]:
def main():
    display(Markdown(
        "мяу мяу, что выбираем?\n\n"
        "[1] демо на нескольких матрицах\n\n"
        "[2] решить свою матрицу A (ручной ввод)\n\n"
        "[3] решить свою матрицу A и задать $\\lambda^*$ вручную\n"
    ))
    mode = (input("выбор (1/2/3): ").strip() or "1")

    if mode == "1":
        n = int(input("n (по умолчанию 6): ").strip() or "6")
        demo_three_inputs_inverse(n=n)
        return

    if mode in {"2", "3"}:
        try:
            from meow import _read_matrix
            A = _read_matrix()
        except Exception:
            A = _read_matrix_fallback()

        Aarr = _as_array_M(A)
        if Aarr.shape[0] != Aarr.shape[1]:
            raise _err("нужно ввести квадратную A")
        n = Aarr.shape[0]

        display(Markdown("выбираем x0: `ones` / `rand` / `e1`..`en`"))
        x0_kind = input("x0 kind → ").strip() or "rand"
        x0 = _make_x0(x0_kind, n, seed=42)

        log_each = (input("печатать каждую итерацию? (y/n, по умолчанию y): ").strip().lower() or "y").startswith("y")
        eps1 = float(input("eps1 (по умолчанию 1e-3): ").strip() or "1e-3")
        eps2 = float(input("eps2 (по умолчанию 1e-6): ").strip() or "1e-6")

        _show_stop_rules_formulas_once()
        stop_fixed = _ask_choice(
            "stop для фиксированных обратных итераций: residual / delta [по умолчанию residual]: ",
            {"residual", "delta"},
            "residual",
        )
        stop_ray = _ask_choice(
            "stop для Рэлеевских обратных итераций: residual / lambda [по умолчанию residual]: ",
            {"residual", "lambda"},
            "residual",
        )

        if mode == "2":
            solve_one_case(
                A, x0, eps_list=(eps1, eps2),
                want_power_lambda_star=True,
                stop_rule_fixed=stop_fixed,
                stop_rule_rayleigh=stop_ray,
                log_each=log_each
            )
            return

        lam_star_manual = input("введи λ* (по умолчанию ρ(x0)): ").strip()

        if lam_star_manual == "":
            lam_star = _rayleigh(_as_array_M(A), _as_array_v(x0).astype(complex))
        else:
            lam_star = complex(lam_star_manual.replace(",", "."))

        solve_one_case(
            A, x0, eps_list=(eps1, eps2),
            want_power_lambda_star=False,
            lambda_star_manual=lam_star,
            log_each=log_each
        )
        return

    print("мяу мяу, выбери 1/2/3")

In [ ]:
main()